In [19]:

import os
import numpy as np
import pandas as pd


class AudioProcessor:
    
    @staticmethod
    def make_dir():
        video_label_path = '/home/expend_toshiba/zhoufengqing/labeled_videos/label'
        video_path = '/home/expend_toshiba/zhoufengqing/labeled_videos/video'
        video_label_list = list(os.walk(video_label_path))[0][2]
        video_list = list(os.walk(video_path))[0][2]
        
        
        os.mkdir('../feature')
        for name in video_label_list:
            os.mkdir('../feature/%s'%name[:-4])
    
    
        
    def FeatureExtraction(self,config,audio,output):
        
        
        if config == 'audiovisual':
            config_path = "/home/opensmile-3.0-linux-x64/config/audiovisual/audio.conf"
            
        elif config == 'mfcc':
            config_path = "/home/opensmile-3.0-linux-x64/config/mfcc/MFCC12_0_D_A_Z.conf"
        elif config == 'emotion':
            config_path = "/home/opensmile-3.0-linux-x64/config/is09-13/IS10_paraling.conf"
        elif config == 'IS09':
            config_path = "/home/opensmile-3.0-linux-x64/config/is09-13/IS09_emotion.conf"
            
        else:
            config_path = config
        os.system("SMILExtract -C %s -I \"%s\" -O \"%s\""%(config_path,audio,output))
        
    
    def process_features(self,source,des_format = 'npy',save=True,save_path = None):
        
        origin_format = source.split('.')[-1]
        
        if origin_format == 'txt':
            with open(source,'r') as f:

                last_line = f.readlines()[-1]
            features=np.array([eval(i) for i in last_line.split(',')[1:-1]])


        elif origin_format=='csv':
            pass
        
        if des_format == 'npy' and save:
            np.save(save_path,features)
        return features
    
    # 批量将音频文件特征提取

    def extract_features_in_batch(self):
        
        
        feature_path = '../feature'
        
        
        audio_path = '../audio'
        audio_games = list(os.walk(audio_path))[0][1]
        
        
        for i,game in enumerate(audio_games):
            game_path = os.path.join(audio_path,game)
            
            # 之后删掉
            round_path_list = list(os.walk(game_path))[0][2]
            
            save_path = os.path.join(feature_path,game)
            
            
            
            for r_audio in round_path_list:
                
                idx, label = r_audio.replace('round','').replace('.wav','').split('_')
                print(r_audio)
                print(idx, label)
                input_path = os.path.join(game_path,r_audio)
                output_path = os.path.join(save_path,r_audio.replace('.wav','.txt'))
                print(output_path)
                self.FeatureExtraction('IS09',input_path,output_path)
                
                self.process_features(output_path,save_path = output_path.replace('.txt','.npy'))
                        
            print("finished:",i)
        
        pass
        
    def compile_data(self):
        
        feature_path = '../feature'
        
        
        audio_path = '../audio'
        audio_games = list(os.walk(audio_path))[0][1]
        df = pd.DataFrame(columns=['game','round','feature','label'])
        
        for i,game in enumerate(audio_games):

        
        
            game_path = os.path.join(audio_path,game)
            round_path_list = list(os.walk(game_path))[0][2]
            save_path = os.path.join(feature_path,game)
            
            
            
            for r_audio in round_path_list:
                
                r = dict()
                r['game'] = game

                idx, label = r_audio.replace('round','').replace('.wav','').split('_')
                
                r['round'] = int(idx)
                r['label'] = int(label)
                r['feature'] = np.load(os.path.join(save_path,r_audio.replace('.wav','.npy')))
                if len(r['feature']) == 0:
                    print('loss ',game,idx,label)
                    continue
                df = df.append(r,ignore_index=True)

            print("finished:",i)
        
        return df        
        
        
        

In [7]:

df.to_pickle('../features.pkl')

In [20]:
p = AudioProcessor()
df=p.compile_data()
df = df.sort_values(by=['game','round','label'])
df.to_pickle('../features.pkl')
df.head()

finished: 0
finished: 1
finished: 2
finished: 3
finished: 4
finished: 5
finished: 6
finished: 7
finished: 8
finished: 9
finished: 10
finished: 11
finished: 12
finished: 13
finished: 14
loss  2018罗马赛女单第二轮贝古VS科贝尔 回放 147 0
finished: 15
finished: 16
finished: 17
finished: 18
finished: 19
finished: 20
finished: 21
finished: 22
finished: 23
finished: 24
finished: 25
finished: 26
finished: 27
finished: 28
finished: 29


,game,round,feature,label
541,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,0,"[0.07375495, 0.00122211, 0.07253284, 193.0, 25...",0
570,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,1,"[0.06153475, 0.001219221, 0.06031553, 190.0, 3...",0
514,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,2,"[0.08575404, 0.0009713328, 0.08478271, 331.0, ...",0
534,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,3,"[0.07541106, 0.001060473, 0.07435059, 241.0, 1...",0
522,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,4,"[0.07332629, 0.001167918, 0.07215837, 364.0, 2...",0


In [12]:
df = df.sort_values(by=['game','round','label'])



In [13]:
df.head()

,game,round,feature,label
541,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,0,"[0.07375495, 0.00122211, 0.07253284, 193.0, 25...",0
570,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,1,"[0.06153475, 0.001219221, 0.06031553, 190.0, 3...",0
514,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,2,"[0.08575404, 0.0009713328, 0.08478271, 331.0, ...",0
534,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,3,"[0.07541106, 0.001060473, 0.07435059, 241.0, 1...",0
522,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,4,"[0.07332629, 0.001167918, 0.07215837, 364.0, 2...",0


In [14]:
p = AudioProcessor()
p.extract_features_in_batch()

round101_0.wav
101 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round101_0.txt
round96_1.wav
96 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round96_1.txt
round100_0.wav
100 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round100_0.txt
round115_0.wav
115 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round115_0.txt
round51_1.wav
51 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round51_1.txt
round95_0.wav
95 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round95_0.txt
round82_0.wav
82 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round82_0.txt
round28_0.wav
28 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round28_0.txt
round76_0.wav
76 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round76_0.txt
round102_1.wav
102 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round102_1.txt
round2_0.wav
2 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round2_0.txt
round33_1.wav
33 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round33_1.txt
round67_1.wav
67 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round67_1.txt
round53_0.wav
53 0
../feature/2018ATP罗杰斯杯男

round44_0.wav
44 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round44_0.txt
round30_1.wav
30 1
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round30_1.txt
round57_0.wav
57 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round57_0.txt
round38_0.wav
38 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round38_0.txt
round45_0.wav
45 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round45_0.txt
round35_0.wav
35 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round35_0.txt
round8_0.wav
8 0
../feature/2018ATP罗杰斯杯男单决赛 纳达尔VS西西帕斯 回放/round8_0.txt
finished: 0
round13_1.wav
13 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round13_1.txt
round17_0.wav
17 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round17_0.txt
round25_0.wav
25 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round25_0.txt
round100_0.wav
100 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round100_0.txt
round98_1.wav
98 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round98_1.txt
round53_1.wav
53 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round53_1.txt
round2_1.wav
2 1
../feature/2018WTA卢加诺赛

round80_0.wav
80 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round80_0.txt
round73_1.wav
73 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round73_1.txt
round69_1.wav
69 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round69_1.txt
round84_0.wav
84 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round84_0.txt
round41_0.wav
41 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round41_0.txt
round66_0.wav
66 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round66_0.txt
round3_1.wav
3 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round3_1.txt
round48_1.wav
48 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round48_1.txt
round30_1.wav
30 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round30_1.txt
round101_1.wav
101 1
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round101_1.txt
round99_0.wav
99 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round99_0.txt
round23_0.wav
23 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round23_0.txt
round57_0.wav
57 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS梅尔滕斯 回放/round57_0.txt
round38_0.wav
38 0
../feature/2018WTA卢加诺赛女单决赛萨巴伦卡VS

round56_0.wav
56 0
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round56_0.txt
round74_1.wav
74 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round74_1.txt
round75_0.wav
75 0
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round75_0.txt
round104_1.wav
104 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round104_1.txt
round50_1.wav
50 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round50_1.txt
round0_1.wav
0 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round0_1.txt
round42_0.wav
42 0
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round42_0.txt
round108_1.wav
108 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round108_1.txt
round19_1.wav
19 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round19_1.txt
round70_1.wav
70 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round70_1.txt
round80_0.wav
80 0
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round80_0.txt
round38_1.wav
38 1
../feature/2018WTA斯特拉斯堡站女单1_4决赛：齐布尔科娃VS斯托瑟 回放/round38_1.txt
round73_1.wav
73 1
../feature/2018WTA斯特拉斯堡站女单1_4决

round94_0.wav
94 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round94_0.txt
round78_0.wav
78 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round78_0.txt
round97_1.wav
97 1
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round97_1.txt
round147_0.wav
147 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round147_0.txt
round43_0.wav
43 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round43_0.txt
round29_0.wav
29 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round29_0.txt
round27_0.wav
27 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round27_0.txt
round85_0.wav
85 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round85_0.txt
round119_0.wav
119 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round119_0.txt
round63_0.wav
63 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round63_0.txt
round106_0.wav
106 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round106_0.txt
round134_0.wav
134 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round134_0.txt
round112_0.wav
112 0
../feature/2018ATP蒙特卡洛赛男单第二轮迪米特洛夫VS赫伯特 回放/round112_0.txt
round150_0

...

round155_0.wav
155 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round155_0.txt
round98_0.wav
98 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round98_0.txt
round120_1.wav
120 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round120_1.txt
round162_1.wav
162 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round162_1.txt
round128_0.wav
128 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round128_0.txt
round148_0.wav
148 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round148_0.txt
round86_1.wav
86 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round86_1.txt
round116_0.wav
116 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round116_0.txt
round111_0.wav
111 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round111_0.txt
round157_0.wav
157 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round157_0.txt
round22_0.wav
22 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round22_0.txt
round15_0.wav
15 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round15_0.txt
round81_0.wav
81 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round81_0.txt
round51_0.wav
51 0
../feature/2018WTA南昌站

round75_0.wav
75 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round75_0.txt
round143_1.wav
143 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round143_1.txt
round42_0.wav
42 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round42_0.txt
round188_0.wav
188 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round188_0.txt
round146_0.wav
146 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round146_0.txt
round24_0.wav
24 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round24_0.txt
round135_1.wav
135 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round135_1.txt
round30_0.wav
30 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round30_0.txt
round68_0.wav
68 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round68_0.txt
round134_1.wav
134 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round134_1.txt
round80_0.wav
80 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round80_0.txt
round150_1.wav
150 1
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round150_1.txt
round66_0.wav
66 0
../feature/2018WTA南昌站女单1_4决赛 张帅VS朱琳 解说/round66_0.txt
round138_1.wav
138 1
../feature/2018WTA南昌站女单1_

round84_0.wav
84 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round84_0.txt
round41_0.wav
41 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round41_0.txt
round66_0.wav
66 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round66_0.txt
round3_1.wav
3 1
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round3_1.txt
round44_0.wav
44 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round44_0.txt
round22_1.wav
22 1
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round22_1.txt
round76_1.wav
76 1
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round76_1.txt
round23_0.wav
23 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round23_0.txt
round57_0.wav
57 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round57_0.txt
round38_0.wav
38 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round38_0.txt
round54_1.wav
54 1
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round54_1.txt
round35_0.wav
35 0
../feature/2018WTA辛辛那提站女单第二轮吉奥尔吉VS凯斯 回放/round35_0.txt
finished: 27
round13_1.wav
13 1
../feature/2019WTA深圳公开赛第6日 萨巴伦卡VS莎拉波娃 英文录播/round13_1.txt
round17_0.wav
17 0
../feature/2019WTA深

round25_0.wav
25 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round25_0.txt
round100_0.wav
100 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round100_0.txt
round124_1.wav
124 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round124_1.txt
round87_1.wav
87 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round87_1.txt
round115_0.wav
115 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round115_0.txt
round98_1.wav
98 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round98_1.txt
round33_0.wav
33 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round33_0.txt
round139_1.wav
139 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round139_1.txt
round51_1.wav
51 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round51_1.txt
round88_1.wav
88 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round88_1.txt
round95_0.wav
95 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round95_0.txt
round132_0.wav
132 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round132_0.txt
round39_0.wav
39 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回

round137_0.wav
137 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round137_0.txt
round93_1.wav
93 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round93_1.txt
round86_0.wav
86 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round86_0.txt
round107_0.wav
107 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round107_0.txt
round122_0.wav
122 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round122_0.txt
round20_0.wav
20 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round20_0.txt
round61_1.wav
61 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round61_1.txt
round55_0.wav
55 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round55_0.txt
round83_0.wav
83 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round83_0.txt
round96_0.wav
96 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round96_0.txt
round85_1.wav
85 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round85_1.txt
round56_0.wav
56 0
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/round56_0.txt
round74_1.wav
74 1
../feature/2018WTA伯明翰站女单半决赛 斯特里科娃VS莱巴里科娃 回放/roun

In [74]:
audio = "/home/expend_toshiba/xieyuning/sample2.wav"
output = "/home/expend_toshiba/xieyuning/sample_emotion092.txt"
p = AudioProcessor()

p.FeatureExtraction('IS09',audio,output)

f2 = p.process_features('/home/expend_toshiba/xieyuning/sample_emotion09.txt')



In [ ]:
os.system("SMILExtract -C /home/opensmile-3.0-linux-x64/config/audiovisual/audio.conf -A /home/expend_toshiba/xieyuning/sample.wav -P /home/expend_toshiba/xieyuning/samples.txt")

In [17]:
dff = pd.read_pickle('../features.pkl')



In [18]:
dff.head()

,game,round,feature,label
541,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,0,"[0.07375495, 0.00122211, 0.07253284, 193.0, 25...",0
570,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,1,"[0.06153475, 0.001219221, 0.06031553, 190.0, 3...",0
514,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,2,"[0.08575404, 0.0009713328, 0.08478271, 331.0, ...",0
534,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,3,"[0.07541106, 0.001060473, 0.07435059, 241.0, 1...",0
522,2018ATP上海大师赛男单半决赛 兹维列夫VS德约科维奇 解说,4,"[0.07332629, 0.001167918, 0.07215837, 364.0, 2...",0
